In [ ]:
## Importing the necessary libraries

In [ ]:
import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

In [ ]:
## preparing the training dataset as well as validation dataset

In [ ]:
data_dir = r'C:\Users\gopal\Downloads\final_year_project\Indian_Medicinal_Leaves_Image_Datasets\Medicinal_Leaf_dataset\train'
classes = [img_folder.name for img_folder in os.scandir(data_dir) if img_folder.is_dir()]

# for cl in classes:
#   imgs_path = os.path.join(data_dir, cl) 
#   imgs = glob.glob(imgs_path + '/*.jpg')
#   print(f"{cl}: {len(imgs)} Images")
#   train, val = imgs[:round(len(imgs) * 0.8)], imgs[round(len(imgs) * 0.8):] 

#   for t in train:
#     if not os.path.exists(os.path.join(data_dir, 'train', cl)): 
#       os.makedirs(os.path.join(data_dir, 'train', cl)) 
#     shutil.move(t, os.path.join(data_dir, 'train', cl))

#   for v in val:
#     if not os.path.exists(os.path.join(data_dir, 'val', cl)): 
#       os.makedirs(os.path.join(data_dir, 'val', cl)) 
#     shutil.move(v, os.path.join(data_dir, 'val', cl))


In [ ]:
## Paths to training and validation datasets

In [ ]:
train_dir = data_dir
val_dir = r'C:\Users\gopal\Downloads\final_year_project\Indian_Medicinal_Leaves_Image_Datasets\Medicinal_Leaf_dataset\val'

In [ ]:
BATCH_SIZE = 100
IMG_SHAPE = 299

In [ ]:
## Display the images in the dataset before augmentation

In [ ]:
def PlotImages(imgs):
  dim = 3 #for a 3x3 of images
  plt.figure(figsize=(10, 10))
  for i in range(dim**2):
    plt.subplot(dim, dim, i+1)
    plt.imshow(imgs[i])

In [ ]:
## Code generates augmented image data for training a neural network model

In [ ]:
img_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    horizontal_flip=True,
    zoom_range=0.5,
    height_shift_range=0.15,
    width_shift_range=0.15,
    brightness_range=(0.5, 1.5),
    fill_mode='wrap'    
)

train_data_gen = img_gen_train.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='categorical'
)

class_indices = train_data_gen.class_indices

class_mapping = {v: k for k, v in class_indices.items()}
print(class_mapping)

In [ ]:
## Visualizing the augumented data 

In [ ]:
# viz_imgs = [train_data_gen[0][0][0] for i in range(9)]
# PlotImages(viz_imgs)

In [ ]:
img_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = img_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=val_dir,
                                               target_size=(IMG_SHAPE, IMG_SHAPE),
                                               class_mode='categorical')

In [ ]:
## Training the neural network model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SHAPE, IMG_SHAPE, 3))

for i in base_model.layers:
    i.trainable = False   

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(len(classes), activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // BATCH_SIZE,
    epochs=50,
)

In [ ]:
## Plotting the accuracy and validation graph

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 4))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)
model.summary()

In [ ]:
## Saves the model to the local system

In [ ]:
model.save('image_classification.keras')